# CMU Movie data

## Downloading datasets
The following code dowloads the datasets that were too heavy to be included, and places them directly in the `data` folder

In [ ]:
#Load functions
from src.utils.import_TMDb import import_TMDb
from src.utils.get_IMDb_dataset import get_IMDb_dataset


#Download TMDb Dataset
import_TMDb()
#Download IMDb Dataset
get_IMDb_dataset("name.basics.tsv")
get_IMDb_dataset("title.ratings.tsv")
get_IMDb_dataset("title.principals.tsv")



## Initial data inspection
We will first try to provide a first generic inspection of the CMU movie dataset we decided to work on.

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import os
from src.utils.data_utils import *
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
%load_ext autoreload
%autoreload 2

### Load Data
The dataset is divided in 3 parts, the characters, the movies and the plots of the movies.

In [ ]:
character_data, movie_data, plot_data = load_data()

### Characters dataset
Let's first have a look to the character dataset.

In [ ]:
print(f'There are {character_data.shape[0]} characters with {character_data.shape[1]} features for each.')

In [ ]:
character_data.head(2)

Note that we have mapped the ethnicity id to a real label (see freebase.py for details on how we got the map).

Let's see if we have a lot of missing data. We will also check that we don't have duplicated rows.

In [ ]:
print("Percentage of null rows in the characters dataset for each features:")
print(character_data.isnull().mean().round(3)*100)

In [ ]:
print(f"Duplicated rows: {character_data.duplicated().sum()}")

We see that we miss a lot of character names/ids, actor heights, ethnicity and age at release.

### Movies dataset
Let's now have a look at the movies dataset.

In [ ]:
print(f'There are {movie_data.shape[0]} movies with {movie_data.shape[1]} features for each.')

In [ ]:
movie_data.head(2)

We can note that the languages, countries and genres need to be preprocessed (for now dictionnary with id->readablename).

Let's now see if we have a lot of missing data. We will also verify that we don't have duplicated rows.

In [ ]:
print("Percentage of null rows in the movies dataset for each features:")
print(movie_data.isnull().mean().round(3)*100)

In [ ]:
print(f"Duplicated rows: {movie_data.duplicated().sum()}")

Ouch! We only have box office revenue for 10% of our movies, that's not good news since it's a key feature in our research problematic, we will need to fix this. Apart from this, we can also note that we are missing 25% of the runtime information. We could try to improve this. This also applies to the movie release data. For the languages, countries and genres, we note that they are dictionaries meaning that we first need to preprocess them a bit (for example transforming them to a list) to then be able to see the percentage of missing data. We will do it now:

In [ ]:
# Extract the readable values for 'languages', 'countries', and 'genres' columns. Also clean the language column.

movie_data['languages'] = movie_data['languages'].apply(lambda x: extract_values(x, clean_func=clean_language))
movie_data['countries'] = movie_data['countries'].apply(lambda x: extract_values(x)) 
movie_data['genres'] = movie_data['genres'].apply(lambda x: extract_values(x))  

We can now have a look to the missing data:

In [ ]:
# Calculate the number of None (NaN) values for each column
none_languages = movie_data['languages'].isna().mean()
none_countries = movie_data['countries'].isna().mean()
none_genres = movie_data['genres'].isna().mean()

# Print the counts of None (NaN) values
print(f"Percentage of None values in 'languages': {none_languages:.2%}")
print(f"Percentage of None values in 'countries': {none_countries:.2%}")
print(f"Percentage of None values in 'genres': {none_genres:.2%}")

This looks ok overall.

### Plot summary dataset
Let's now have a look at the plot summaries dataset.

In [ ]:
print(f'There are {plot_data.shape[0]} plot summaries with {plot_data.shape[1]} features for each.')

In [ ]:
plot_data.head(2)

Let's see if we have some rows that are invalid (no summary or wikipedia id).

In [ ]:
print("Pourcentage of null rows in the plot summaries dataset:")
print(plot_data.isnull().mean().round(3)*100)

Good new, we have nothing missing here :)

## Data completion
Before going deeper to the analysis, we want to already fix the fact that we are missing data for most of the box office results and some runtime values.

Let's then first to get more data on box office results to decrease the number of missing data we have for now. To do this, we will merge the current dataset with a dataset that contains information about 1,000,000 movies collected from The Movie Database (TMDb), including revenue and runtime (https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies?resource=download).

In [ ]:
# Load the new dataset (and rename some columns)
movies_dataset = pd.read_csv('data/tmbd_movies.csv')
movies_dataset['box_office_revenue'] = pd.to_numeric(movies_dataset['revenue'], errors='coerce') 
movies_dataset['release_date'] = pd.to_datetime(movies_dataset['release_date'], errors='coerce')

# We remove the nan of movie release date since we merge on it
movie_data['movie_release_date'] = pd.to_datetime(movie_data['movie_release_date'], errors='coerce')
movie_data = movie_data.dropna(subset=['movie_release_date'])

movies_dataset.head(2)

In [ ]:
movie_data_merged, before_missing, after_missing = merge_for_completion(movie_data, movies_dataset, ["movie_name", "movie_release_date"], ["title", "release_date"], "box_office_revenue", merge_strategy='prioritize_first')

In [ ]:
# Print the before and after missing percentages
print(f"Box office results missing percentage before merge (on title) with wikidata: {before_missing:.2%}")
print(f"Box office results missing percentage after merge (on title) with wikidata: {after_missing:.2%}")

Big improvement, that's good. Let's try to improve the runtime data.

In [ ]:
movie_data_merged, before_missing, after_missing = merge_for_completion(movie_data_merged, movies_dataset, ["movie_name", "movie_release_date"], ["title", "release_date"], "runtime", merge_strategy='prioritize_first')

In [ ]:
# Print the before and after missing percentages
print(f"Runtime results missing percentage before merge (on title) with The Movies Dataset: {before_missing:.2%}")
print(f"Runtime results missing percentage after merge (on title) with The Movies Dataset: {after_missing:.2%}")

That's pretty cool too, we also want to get the IMBD ID column because we need it afterwards.

In [ ]:
movie_data_merged, _, _ = merge_for_completion(movie_data_merged, movies_dataset, ["movie_name", "movie_release_date"], ["title", "release_date"], "imdb_id", merge_strategy='add_column')

## Getting the rating and lead actors of movies

Now we want to merge with the IMDb datasets (https://developer.imdb.com/non-commercial-datasets/) in order to obtain ratings and lead actors. We consider an actor to be a lead actor if their ordering is 1 or 2.

In [ ]:
# Load IMDb ratings and select relevant columns
imdb_ratings = pd.read_csv('data/title.ratings.tsv', sep='\t')
imdb_ratings = imdb_ratings.rename(columns={'tconst': 'imdb_id'})
imdb_ratings = imdb_ratings[['imdb_id', 'averageRating', 'numVotes']]

In [ ]:
# Load IMDb names data for actors
imdb_names = pd.read_csv('data/name.basics.tsv', sep='\t')

In [ ]:
# Initialize an empty list to hold chunks of filtered lead actors
filtered_lead_actors = []

# Process imdb_principals in chunks to reduce memory usage
for chunk in pd.read_csv('data/title.principals.tsv', sep='\t', chunksize=100000):
    # Filter for lead actors (first and second-billed actor or actress)
    chunk_lead_actors = chunk[
        (chunk['category'].isin(['actor', 'actress'])) & 
        (chunk['ordering'].isin([1, 2]))
    ][['tconst', 'nconst', 'ordering']]
    
    # Append the filtered chunk to the list
    filtered_lead_actors.append(chunk_lead_actors)

In [ ]:
# Concatenate all filtered chunks into a single DataFrame
lead_actors = pd.concat(filtered_lead_actors)

# Rename columns and merge with imdb_names DataFrame to get actor names
lead_actors = lead_actors.rename(columns={'tconst': 'imdb_id'})
lead_actors = lead_actors.merge(imdb_names[['nconst', 'primaryName']], on='nconst', how='left')

# Pivot to get separate columns for the first and second lead actors
lead_actors = lead_actors.pivot(index='imdb_id', columns='ordering', values='primaryName').reset_index()
lead_actors.columns = ['imdb_id', 'lead_actor_1', 'lead_actor_2']

# Merge movies_wikidata_merged_imdbid with IMDb ratings
imdb_merged_movie_data = pd.merge(movie_data_merged, imdb_ratings, on='imdb_id', how='left')

# Merge with lead actors data
merged_movie_data = imdb_merged_movie_data.merge(lead_actors, on='imdb_id', how='left')

In [ ]:
merged_movie_data = merged_movie_data.drop_duplicates(subset=['imdb_id'])
merged_movie_data.head(2)

In [ ]:
print(f"We have {merged_movie_data.shape[0]} rows in our merged movies dataset.")

## Cleaning and removing outliers

Before analyzing the data any further, let's remove outliers.

### Character dataset

In [ ]:
# Convert dob and movie_release_date to date
character_data.loc[:, "actor_dob"] = pd.to_datetime(character_data["actor_dob"], errors='coerce')
character_data.loc[:, "movie_release_date"] = pd.to_datetime(character_data["movie_release_date"], errors='coerce')


# Create a mask for cases where `actor_dob` and `movie_release_date` are present but `actor_age_at_release` is missing
missing_age_cases = character_data['actor_age_at_release'].isna() & \
                    character_data['actor_dob'].notna() & \
                    character_data['movie_release_date'].notna()
# Do some of this cases exist?
print(f"before adding ages: {character_data.loc[:,"actor_age_at_release"].isnull().count()}")
print(missing_age_cases.any())
print(f"This amount of ages can be retrieved through calculation:{character_data.loc[missing_age_cases,:].shape[0]}")

# In this case we can calculate the actor age at release
for i in character_data[missing_age_cases].index:
    character_data.loc[i,'actor_age_at_release']=character_data.loc[i,'movie_release_date'].year - character_data.loc[i,'actor_dob'].year
print(f"after adding ages: {character_data["actor_age_at_release"].isna().count()}")
character_data.loc[missing_age_cases,"actor_age_at_release"].head()

In [ ]:
# Keep only non-NaN values for all columns
columns_names = ['actor_height', 'actor_age_at_release', 'actor_gender', 
                 'actor_ethnicity_label', 'character_name', 'actor_dob']
character_data_cleaned, reduction = keep_only_non_nans(character_data, columns_names)

print(f"Removing NaN reduced the dataset by: {reduction:.2%}")

In [ ]:
# Keep only valid heights (between 1.5 and 2.8 meters)
character_data_valid_heights, reduction = filter_attribute(character_data_cleaned, 'actor_height', 1.5, 2.8)

print(f"Removing invalid actor heights reduced that dataset by {reduction:.2%}.")

In [ ]:
# Keep only valid ages (between 0 and 100 years)
character_data_valid_ages, reduction = filter_attribute(character_data_valid_heights, 'actor_age_at_release', 0, 100)

print(f"Removing invalid actor ages reduced that dataset by {reduction:.2%}.")

In [ ]:
# Keep only ethnicity labels that are common
min_occurrence = 10
ethnicity_label_counts = character_data_valid_ages['actor_ethnicity_label'].value_counts()
ethnicity_labels = ethnicity_label_counts[ethnicity_label_counts > min_occurrence]

mask = character_data_valid_ages['actor_ethnicity_label'].isin(ethnicity_labels.index)
character_data_valid = character_data_valid_ages[mask]

reduction = compute_reduction(character_data_valid_ages, character_data_valid)

print(f"Removing ethnicity labels which are uncommon reduced that dataset by {reduction:.2%}.")

In [ ]:
# Print the final dataset size
print(f"Number of characters after preprocessing: {character_data_valid.shape[0]}.")

In [ ]:
# Check that no nan values still there
character_data_valid.isnull().mean()

### Movies dataset

We will also remove the outliers and preprocess the movie dataset.

In [ ]:
# Drop duplicated columns
movie_data_extracted = merged_movie_data.drop(columns=['title', 'release_date'])

In [ ]:
# Set lead_actor_2 to NaN where it is the same as lead_actor_1
movie_data_extracted.loc[movie_data_extracted['lead_actor_1'] == movie_data_extracted['lead_actor_2'], 'lead_actor_2'] = pd.NA

# Remove movies where we don't have lead actor
movie_data_extracted = movie_data_extracted.dropna(subset=['lead_actor_1'])

In [ ]:
# Keep only non-NaN values for all columns (the other columns have no missing values)
columns_names = ['movie_release_date', 'runtime', 'languages', 'countries', 
                 'genres', 'lead_actor_1', 'box_office_revenue', 'averageRating']
movie_data_cleaned, reduction = keep_only_non_nans(movie_data_extracted, columns_names)

print(f"Removing NaN reduced the dataset by: {reduction:.2%}")

In [ ]:
#Keep only movies released after 1940
movie_data_valid_release_dates = movie_data_cleaned[movie_data_cleaned["movie_release_date"]>'1-1-1940']

reduction = compute_reduction(movie_data_cleaned, movie_data_valid_release_dates)
print(f"Removing movies released before 1940 reduced the dataset by: {reduction:.2%}")

In [ ]:
# Keep only movies that last between 1h and 200min
movie_data_valid_runtime, reduction = filter_attribute(movie_data_valid_release_dates, 'runtime', 60, 200)
print(f"Removing movies lasting less than 1h or more than 3 hours 20mins reduced the dataset by: {reduction:.2%}")

In [ ]:
#Keep only movies that have at least 500 votes
movie_data_valid_votes = movie_data_valid_runtime[movie_data_valid_runtime["numVotes"]>500]

reduction = compute_reduction(movie_data_valid_runtime, movie_data_valid_votes)
print(f"Removing movies that have less than 500 votes: {reduction:.2%}")

In [ ]:
# Keep only movies that have more than 0 box office revenue
movie_data_valid_revenue = movie_data_valid_votes[movie_data_valid_votes["box_office_revenue"]>0]

reduction = compute_reduction(movie_data_valid_votes, movie_data_valid_revenue)
print(f"Removing movies that have no box office revenue: {reduction:.2%}")

In [ ]:
movie_data_valid = movie_data_valid_revenue.copy()

In [ ]:
print(f"Number of movies after preprocessing: {movie_data_valid.shape[0]}.")

In [ ]:
# Check that no nan values still there
movie_data_valid.isnull().mean()

## Do we have data for the lead actors

Now that we have the two main actors of each movie, let's see if we have their characteristics.

In [ ]:
# Extract unique pairs of (freebase_movie_id, lead_actor) from the movie dataset
lead_actor_pairs = pd.concat([
    movie_data_valid[['freebase_movie_id', 'lead_actor_1']].rename(columns={'lead_actor_1': 'actor_name'}),
    movie_data_valid[['freebase_movie_id', 'lead_actor_2']].rename(columns={'lead_actor_2': 'actor_name'})
])

# Convert the DataFrame of pairs to a list of tuples for filtering
lead_actor_pairs = list(lead_actor_pairs.itertuples(index=False, name=None))

# Filter character_data to keep only rows where (freebase_movie_id, actor_name) matches the pairs in lead_actor_pairs
lead_actor_data = character_data_valid[
    character_data_valid[['freebase_movie_id', 'actor_name']].apply(tuple, axis=1).isin(lead_actor_pairs)
]

# Check for missing values in key columns
print("Missing values in lead actor data:")
print(lead_actor_data[['actor_name', 'actor_dob', 'actor_gender', 'actor_ethnicity', 'actor_height', 'actor_age_at_release']].isna().mean()*100)

# Display the first few rows of the filtered data
lead_actor_data.head(2)


This make sense since we preprocessed our character data, let's now see if we have data for all of our actors.

In [ ]:
print(f"We have data for {lead_actor_data.shape[0]/(movie_data_valid["lead_actor_1"].notna().sum() + movie_data_valid["lead_actor_2"].notna().sum())*100}% ({lead_actor_data.shape[0]} actors) of our lead actors")

Ok good, let's now create a subset dataframe that will contains only the movies for which we have data on the lead actors.

In [ ]:
subset_movies_with_lead_actors_data = movie_data_valid[movie_data_valid['freebase_movie_id'].isin(lead_actor_data['freebase_movie_id'])]

In [ ]:
print(f"We have {subset_movies_with_lead_actors_data.shape[0]} movies for which we have full data on our lead actors")

We can also extract the characters that are part of this subset.

In [ ]:
# Filter the character_data_valid dataset to keep only rows with freebase_movie_id present in movie_data_extracted
subset_characters_with_lead_actor_data = character_data_valid[character_data_valid['freebase_movie_id'].isin(subset_movies_with_lead_actors_data['freebase_movie_id'])]

# Extract the relevant columns for characters and associated movies
subset_characters_with_lead_actor_data = subset_characters_with_lead_actor_data[['actor_name', 'actor_dob', 'actor_gender', 'actor_ethnicity', 'actor_height', 'actor_age_at_release', 'freebase_movie_id', 'character_name']]

# Display the first few rows of the filtered and extracted data
subset_characters_with_lead_actor_data.head(2)


In [ ]:
print(f"We have {subset_characters_with_lead_actor_data.shape[0]} characters for the subset movies.")

## Saving our newly created dataframes

We will save our five dataframes:

- full_movie_data_preprocessed that contains all the movies preprocessed 
- full_characters_data_preprocessed that contains all the characters preprocessed
- subset_movies_with_lead_actors_data that contains movies where we have data on all the lead actors
- lead_actors_data_on_subset_movie that contains information on the lead actors of the previous subset
- character_data_valid_filtered that contains information on all the characters of the previous subset

In [ ]:
preprocessed_path = 'data/preprocessed'

# Ensure the directory exists
os.makedirs(preprocessed_path, exist_ok=True)

preprocessed_names = {
    'full_movie_data_preprocessed.csv': movie_data_valid,
    'full_characters_data_preprocessed.csv': character_data_valid,
    'subset_movie_with_full_data_on_lead_actors.csv': subset_movies_with_lead_actors_data,
    'lead_actors_data_on_subset_movie.csv': lead_actor_data,
    'characters_data_on_subset_movie.csv': subset_characters_with_lead_actor_data
}

for name, df in preprocessed_names.items():
    df.to_csv(os.path.join(preprocessed_path, name), index=False)

## Loading our preprocessed dataframes

We can skip all the preprocessing process and load our dataframes directly here:

In [ ]:
# Load the preprocessed datasets
full_movie_data_preprocessed = pd.read_csv('data/preprocessed/full_movie_data_preprocessed.csv')
full_characters_data_preprocessed = pd.read_csv('data/preprocessed/full_characters_data_preprocessed.csv')
subset_movie_with_full_data_on_lead_actors = pd.read_csv('data/preprocessed/subset_movie_with_full_data_on_lead_actors.csv')
lead_actors_data_on_subset_movie = pd.read_csv('data/preprocessed/lead_actors_data_on_subset_movie.csv')
characters_data_on_subset_movie = pd.read_csv('data/preprocessed/characters_data_on_subset_movie.csv')

# We must convert dates to datetime
lead_actors_data_on_subset_movie['actor_dob'] = pd.to_datetime(lead_actors_data_on_subset_movie['actor_dob'])
characters_data_on_subset_movie['actor_dob'] = pd.to_datetime(characters_data_on_subset_movie['actor_dob'])
full_movie_data_preprocessed['movie_release_date'] = pd.to_datetime(full_movie_data_preprocessed['movie_release_date'])


## Deeper analysis
Now that our data is more complete, we can do a more in deep analysis.

### Lead actors dataset 

Let's first analyze our dataframe with the lead actors just created. We will start with a summary of the statistics of the numerical features.

In [ ]:
lead_actors_data_on_subset_movie.describe()

Let's print their distributions:

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

# Histogram for the height of actors
sns.histplot(data=lead_actors_data_on_subset_movie, x="actor_height", bins=25, ax=axes[0], kde=False)
axes[0].set_title("Height of the actor")
axes[0].set_xlabel("Height (m)")

# Histogram for the actors' age at release
sns.histplot(data=lead_actors_data_on_subset_movie, x="actor_age_at_release", bins=25, ax=axes[1], kde=True)
axes[1].set_title("Age of the actor at the release of the movie")
axes[1].set_xlabel("Age (years)")

# Histogram for the character date of birth
sns.histplot(data=lead_actors_data_on_subset_movie, x="actor_dob", bins=25, ax=axes[2], kde=True)
axes[2].set_title("Date of birth of the actor")
axes[2].set_xlabel("Date of birth")

plt.tight_layout()
plt.show()


Let's now lets explore the categorical data.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Countplot for the gender distribution
sns.countplot(data=lead_actors_data_on_subset_movie, x="actor_gender", ax=axes[0], stat='proportion')
axes[0].set_title("Actor gender distribution")
axes[0].set_xlabel("Gender")
axes[0].set_ylabel("Proportion")

ethnicity_cutoff = 30
values = lead_actors_data_on_subset_movie["actor_ethnicity_label"].value_counts()
values = values[:ethnicity_cutoff]
sns.barplot(x=values, y=values.index, ax=axes[1])
axes[1].set_title(f"{ethnicity_cutoff} most common ethnicity label")
axes[1].set_xlabel("Count")
axes[1].set_ylabel("Ethnicity")

plt.tight_layout()
plt.show()

We can visualize here the correlation matrix between the numerical attributes of the actors.

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

lead_actors_for_corr = lead_actors_data_on_subset_movie[['actor_height', 'actor_age_at_release','actor_gender' ]]
# Convert gender to boolean by getting dummies
lead_actors_gender_dummies = pd.get_dummies(lead_actors_for_corr['actor_gender'])
lead_actors_for_corr.drop('actor_gender', axis=1, inplace=True)
lead_actors_for_corr = lead_actors_for_corr.join(lead_actors_gender_dummies)
# lead_actors_for_corr['F'] = lead_actors_gender_dummies['F']
# lead_actors_for_corr['M'] = lead_actors_gender_dummies['M']
corr_matrix = lead_actors_for_corr.corr()
label_names = ['Height', 'Age at release']
ax = sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', xticklabels=label_names, yticklabels=label_names)
ax.set_title('Heatmap of some actors attributes');


### Movies dataset 

Let's now analyse our movies dataset. We will start with a summary of the statistics of the numerical features.

In [ ]:
movie_data_completed = full_movie_data_preprocessed.copy()
movie_data_completed.describe()

Let's print their distributions (except for the wikipedia id):

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

# Histogram for the runtime
sns.histplot(data=movie_data_completed, x="runtime", bins=50, ax=axes[0], kde=True)
axes[0].set_title("Histogram for runtime")
axes[0].set_xlabel("Runtime (min)")

# Histogram for the box office results
sns.histplot(data=movie_data_completed, x="box_office_revenue", bins=50, ax=axes[1], kde=True, log_scale=True)
axes[1].set_title("Histogram for the box office revenue")
axes[1].set_xlabel("Box office revenue (dollars, log scale)")


# Histogram for the release date
sns.histplot(data=movie_data_completed, x="movie_release_date", bins=50, ax=axes[2], kde=False)
axes[2].set_title("Histogram for the release date")
axes[2].set_xlabel("Release date")

plt.tight_layout()
plt.show()


Let's now look at the ratings and number of votes per movie.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Histogram for the movie ratings
rating_intervals = np.arange(0, 11, 1)
sns.histplot(data=movie_data_completed, x="averageRating", bins=rating_intervals, ax=axes[0])

axes[0].set_xlabel('Rating intervals')
axes[0].set_xticks(rating_intervals)
axes[0].set_title('Histogram for movie ratings')

# Histogram for the number of votes
sns.histplot(data=movie_data_completed, x="numVotes", bins=50, ax=axes[1], kde=True, log_scale=True)

axes[1].set_xlabel('Number of votes (log scale)')
axes[1].set_xscale('log')
axes[1].set_title('Histogram for number of votes')

plt.tight_layout()
plt.show()

In [ ]:
movie_data_completed.head()

We can now have a look at the categorical features:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
cutoff = 20 

# Flatten the columns containing the lists of languages
movie_languages_flat = movie_data_completed['languages'].apply(eval).explode()
values = movie_languages_flat.value_counts()
values = values[:cutoff]

# Barplot for the languages distribution
sns.barplot(x=values, y=values.index, ax=axes[0])
axes[0].set_title(f"{cutoff} most common languages")
axes[0].set_xlabel("Count")
axes[0].set_ylabel("Language")

# Flatten the columns containing the lists of countries
movie_countries_flat = movie_data_completed['countries'].apply(eval).explode()
values = movie_countries_flat.value_counts()
values = values[:cutoff]

# Barplot for the countries distribution
sns.barplot(x=values, y=values.index, ax=axes[1])
axes[1].set_title(f"{cutoff} most common countries")
axes[1].set_xlabel("Count")
axes[1].set_ylabel("Country")

plt.tight_layout()
plt.show()

As we can expect, the English language and United States of America clearly dominate the movie industry.

Let's now look at the most common genres that appear in our dataset.

In [ ]:
genres_cutoff = 30

# Flatten the columns containing the lists of genres
movie_genres_flat = movie_data_completed['genres'].apply(eval).explode()
values = movie_genres_flat.value_counts()
values = values[:genres_cutoff]

# Countplot for the languages distribution
fig, ax = plt.subplots(figsize=(8, 6))
sns.barplot(x=values, y=values.index, ax=ax)
ax.set_xlabel('Count')
ax.set_ylabel('Genres')
ax.set_title(f'{genres_cutoff} most common genres');

fig.tight_layout()

### Characters dataset

We now analyze the dataframe with the characters (including non-lead actors). We again start with a summary of the statistics of the numerical features.

In [ ]:
characters_data_on_subset_movie.describe()

Let's do some plots to visualize better. As it is very similar as the plots from the lead actors dataset, we will compare our plots with the ones from this dataset.

In [ ]:
# Define label variables to ensure uniform legends in the plots
label_all = "All actors"
label_lead = "Lead actors"

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 6))

# Histogram for the height of actors
sns.histplot(data=characters_data_on_subset_movie, x="actor_height", bins=25, ax=axes[0], 
             kde=False, stat='percent', label=label_all)
sns.histplot(data=lead_actors_data_on_subset_movie, x="actor_height", bins=25, 
             ax=axes[0], kde=False, stat='percent', label=label_lead)
axes[0].set_title("Height of the actor")
axes[0].set_xlabel("Height (m)")
axes[0].legend()

# Histogram for the age of actors at release
sns.histplot(data=characters_data_on_subset_movie, x="actor_age_at_release", bins=25, ax=axes[1], 
             kde=True, stat='percent', label=label_all)
sns.histplot(data=lead_actors_data_on_subset_movie, x="actor_age_at_release", bins=25, ax=axes[1], 
             kde=True, stat='percent', label=label_lead)
axes[1].set_title("Age of the actor at the release of the movie")
axes[1].set_xlabel("Age (years)")
axes[1].legend()

# Histogram for the character date of birth
sns.histplot(data=characters_data_on_subset_movie, x="actor_dob", bins=25, ax=axes[2], 
             kde=True, stat='percent', label=label_all)
sns.histplot(data=lead_actors_data_on_subset_movie, x="actor_dob", bins=25, ax=axes[2],
             kde=True, stat='percent', label=label_lead)
axes[2].set_title("Date of birth of the actor")
axes[2].set_xlabel("Date of birth")
axes[2].legend()

plt.tight_layout()
plt.show()


Finally we compare the gender distribution between all actors and only lead actors.

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
width = 0.4

gender_dist_all = characters_data_on_subset_movie['actor_gender'].value_counts(normalize=True) * 100
gender_dist_lead = lead_actors_data_on_subset_movie['actor_gender'].value_counts(normalize=True) * 100
labels = list(gender_dist_all.keys())

bar_all = ax.bar(labels, gender_dist_all, -width, label=label_all, align='edge')
bar_lead = ax.bar(labels, gender_dist_lead, width, label=label_lead, align='edge')

ax.set_title("Comparison of actors' gender distribution between lead roles and all roles")
ax.set_xlabel("Gender")
ax.set_ylabel("Percent")
plt.grid(axis='y', linestyle='--', alpha=0.5)

ax.legend()
plt.show()

### Regression analysis

Let's see if we can predict the box office from the average rating and the number of votes!

In [ ]:
# Only keep the box office revenue that is greater than 0
valid_box_office = full_movie_data_preprocessed["box_office_revenue"] > 0
full_movie_data_preprocessed = full_movie_data_preprocessed[valid_box_office]

# Take the log of the box office revenue and of numVotes
full_movie_data_preprocessed["log_box_office_revenue"] = np.log(full_movie_data_preprocessed["box_office_revenue"])
full_movie_data_preprocessed["log_numVotes"] = np.log(full_movie_data_preprocessed["numVotes"])

In [ ]:
# Define the model formula
mod = smf.ols(formula='log_box_office_revenue ~ averageRating + log_numVotes', data=full_movie_data_preprocessed)

# Set a random seed for reproducibility
np.random.seed(0)

# Fit the model
res = mod.fit()

# Print the summary
print(res.summary())

Interesting, there is a strong positive corelation between the number of votes and the box office revenue.
Let's visualize this in a scatter plot!

In [ ]:
sns.scatterplot(data=full_movie_data_preprocessed, x='averageRating', y='log_numVotes', hue='log_box_office_revenue')